In [425]:
import pandas as pd
import numpy as np


In [426]:
df = pd.read_csv('jobOffers.csv', index_col=0, skipinitialspace=True)

In [427]:
#salary
df = df.drop(df[df['salaryRange'] == 'neplacená stáž'].index)
df['salLower'] = df['salaryRange'].str.split(" - ").str[0].str.split("  ").str[0]
df['salUpper'] = df['salaryRange'].str.split(" - ").str[1].str.split("  ").str[0]

df['salLower'] = df['salLower'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)
df['salUpper'].fillna(df['salLower'], inplace = True)
df['salUpper'] = df['salUpper'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

df['salAvg'] = (df['salUpper'] + df['salLower'])/2

df

,jobTitles,company,salaryRange,Bonus,category+progLang,seniority,workLanguage,location,remote,salLower,salUpper,salAvg
0,Java Software Developer,Intempt Technologies,23.2k - 34.9k CZK,NaN,"Backend, java",Junior,Work language:angličtina,NaN,Na dálku,23200,34900,29050.0
1,C++ Developer,Sourcein,70k - 140k CZK,NaN,"Backend, C#","Mid, Senior",Work language:česky,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Covid-time na dálku,70000,140000,105000.0
2,C# Developer,Sourcein,70k - 140k CZK,NaN,"Backend, C#","Mid, Senior",Work language:česky,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Covid-time na dálku,70000,140000,105000.0
3,Java developer u Medvědů,Coding Bear,60k - 120k CZK,NaN,"Backend, java","Mid, Senior",Work language:česky,"Praha, Vítkova 241/10 • Praha, Vítkova 241/10",NaN,60000,120000,90000.0
4,Senior Backend Developer,DX Heroes,60k - 100k CZK,NaN,"Backend, node","Mid, Senior","Work language:česky, angličtina","Praha, Rohanské nábř. 19 • Remote• Praha, Roh...",Na dálku,60000,100000,80000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
712,Remote Database Administrator,MeWe,139.7k - 186.3k CZK,+5%,"DevOps, kubernetes","Senior, Expert",Work language:angličtina,Los Angeles • Remote• Los Angeles,Na dálku,139700,186300,163000.0
713,Senior DevOps Engineer,Hitachi ABB Power Grids,73.1k - 104.4k CZK,NaN,"DevOps, PowerShell","Senior, Expert","Work language:angličtina, polsky",NaN,Na dálku,73100,104400,88750.0
714,DevOps Engineer,Storypoint,139.7k - 186.3k CZK,NaN,"DevOps, python","Mid, Senior","Work language:ukrajinsky, angličtina",NaN,Na dálku,139700,186300,163000.0
715,DevOps Engineer,Appliscale Sp. z o.o.,75.7k - 151.4k CZK,NaN,"DevOps, ansible","Mid, Senior",Work language:angličtina,"krakow, Życzkowskiego 14 • Remote• krakow, Ży...",Na dálku,75700,151400,113550.0


In [428]:
#category and main programming language
df['category'] = df['category+progLang'].str.split(", ").str[0]
df['mainProgLang'] = df['category+progLang'].str.split(", ").str[1]


In [429]:
#seniority
df["seniority"] = df["seniority"].str.strip().replace({'Trainee, Junior': 'Junior', 'Trainee': 'Junior',
                                                          'Senior, Expert':'Senior',
                                                          'Expert':'Senior', 'Mid, Senior':'Mid Advanced'})


In [430]:
#Work language
df["workLanguage"]=df["workLanguage"].str.replace("Work language:","")


In [ ]:
#getting rid of the numbers, dots and converting to lowercase to standardise prog langugage
df_final.loc[:,"mainProgLang"] = df_final.loc[:,"mainProgLang"].str.replace('\d+', '')
df_final.loc[:,"mainProgLang"] = df_final.loc[:,"mainProgLang"].str.replace('.', '')
df["mainProgLang"]=df["mainProgLang"].str.lower()

In [431]:
#leaving just first adress in location for simplification
df["location"]=df["location"].str.split(",").str[0]
df["location"]=df["location"].str.split("•").str[0]
#adjusting remote with location -- Where is Na dálku in remote => location: Remote, 
#when remote: Nan or Covid-time na dálku => location: first city witout street adress
df.loc[df.remote == "Na dálku ","location"]= "Remote"



In [432]:
#subseting dataset
df_final=df[['jobTitles', 'company', 'Bonus','seniority', 'workLanguage', 'location','salAvg', 'category',
       'mainProgLang']]
df_final.head()


,jobTitles,company,Bonus,seniority,workLanguage,location,salAvg,category,mainProgLang
0,Java Software Developer,Intempt Technologies,NaN,Junior,angličtina,Remote,29050.0,Backend,java
1,C++ Developer,Sourcein,NaN,Mid Advanced,česky,Prague,105000.0,Backend,c#
2,C# Developer,Sourcein,NaN,Mid Advanced,česky,Prague,105000.0,Backend,c#
3,Java developer u Medvědů,Coding Bear,NaN,Mid Advanced,česky,Praha,90000.0,Backend,java
4,Senior Backend Developer,DX Heroes,NaN,Mid Advanced,"česky, angličtina",Remote,80000.0,Backend,node


In [433]:
#do we keep bonus? (and jobTitles and company?)
df_final.groupby("Bonus").size()

Bonus
+1%           1
+10%         25
+12%          1
+13%          1
+15%          6
+2%           1
+20%          2
+25%          1
+30%          2
+5%           5
+50%          4
+7%           1
10000 PLN     1
20k CZK       1
dtype: int64

In [449]:
#making sure there are no whitespaces left
df_final.columns = df_final.columns.str.strip()



In [453]:
#to CSV
df_final.to_csv("cleaned_Ivan.csv")